In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import librosa as lr
import librosa.display
import IPython.display as ipd

In [2]:
data_dir = "./Audio"
audio_files = glob(data_dir + "/*.wav")
audio_files

['./Audio\\03-01-01-01-01-01-01.wav',
 './Audio\\03-01-01-01-01-01-02.wav',
 './Audio\\03-01-01-01-01-01-03.wav',
 './Audio\\03-01-01-01-01-01-04.wav',
 './Audio\\03-01-01-01-01-01-05.wav',
 './Audio\\03-01-01-01-01-01-06.wav',
 './Audio\\03-01-01-01-01-01-07.wav',
 './Audio\\03-01-01-01-01-01-08.wav',
 './Audio\\03-01-01-01-01-01-09.wav',
 './Audio\\03-01-01-01-01-01-10.wav',
 './Audio\\03-01-01-01-01-01-11.wav',
 './Audio\\03-01-01-01-01-01-12.wav',
 './Audio\\03-01-01-01-01-01-13.wav',
 './Audio\\03-01-01-01-01-01-14.wav',
 './Audio\\03-01-01-01-01-01-15.wav',
 './Audio\\03-01-01-01-01-01-16.wav',
 './Audio\\03-01-01-01-01-01-17.wav',
 './Audio\\03-01-01-01-01-01-18.wav',
 './Audio\\03-01-01-01-01-01-19.wav',
 './Audio\\03-01-01-01-01-01-20.wav',
 './Audio\\03-01-01-01-01-01-21.wav',
 './Audio\\03-01-01-01-01-01-22.wav',
 './Audio\\03-01-01-01-01-01-23.wav',
 './Audio\\03-01-01-01-01-01-24.wav',
 './Audio\\03-01-01-01-01-02-01.wav',
 './Audio\\03-01-01-01-01-02-02.wav',
 './Audio\\0

In [3]:
data = pd.DataFrame()
Emotion = []
Emotional_Intensity = []
Statement = []
Statement_Text = []
Repetition = []
Actor = []

In [4]:
for i in range(len(audio_files)):
    temp = audio_files[i][audio_files[i].index("\\")+1:audio_files[0].index(".wav")].split("-")
    
    # creating Emotion
    if temp[2] == "01":
        Emotion.append(1)
    if temp[2] == "02":
        Emotion.append(2)
    if temp[2] == "03":
        Emotion.append(3)
    if temp[2] == "04":
        Emotion.append(4)
    if temp[2] == "05":
        Emotion.append(5)
    if temp[2] == "06":
        Emotion.append(6)
    if temp[2] == "07":
        Emotion.append(7)
    if temp[2] == "08":
        Emotion.append(8)

   ############################

    # creating Emotinal_Intensity
    if temp[3] == "01":
        Emotional_Intensity.append(1)
    if temp[3] == "02":
        Emotional_Intensity.append(2)
        
   #############################

    # creating Statement
    if temp[4] == "01":
        Statement.append(1)
        Statement_Text.append("Kids are talking by the door")
    if temp[4] == "02":
        Statement.append(2)
        Statement_Text.append("Dogs are sitting by the door")
        
    ############################
    
    # creating Repetition
    if temp[5] == "01":
        Repetition.append(1)
    if temp[5] == "02":
        Repetition.append(2)
        
    ###########################
    
    # creating actor gender
    if int(temp[6])%2 == 0:   # even --> female
        Actor.append(1)
    if int(temp[6])%2 == 1:   # odd --> male
        Actor.append(0)   

In [5]:
data["Emotion"] = Emotion
data["Emotional_Intensity"] = Emotional_Intensity
data["Statement"] = Statement
data["Statement_Text"] = Statement_Text
data["Repetition"] = Repetition
data["Actor_Gender"] = Actor

In [43]:
# feature extraction
mfcc = []
chroma_stft = []
chroma_cqt = []
chroma_cens = []
rms = []
spectral_contrast = []
spectral_bandwidth = []
tonnetz = []
zcr = []

for i in range(0,len(audio_files)):
    
    y, sr = lr.load(audio_files[i],res_type="kaiser_fast",duration=3,offset=0.5)
    sr=np.array(sr)
    mfcc.append(np.mean(lr.feature.mfcc(y=y,n_mfcc=25,)))
    chroma_stft.append(np.mean(lr.feature.chroma_stft(y=y)))
    chroma_cqt.append(np.mean(lr.feature.chroma_cqt(y=y)))
    chroma_cens.append(np.mean(lr.feature.chroma_cens(y=y)))
    rms.append(np.mean(lr.feature.rms(y=y)))
    spectral_contrast.append(np.mean(lr.feature.spectral_contrast(y=y)))
    spectral_bandwidth.append(np.mean(lr.feature.spectral_bandwidth(y=y)))
    tonnetz.append(np.mean(lr.feature.tonnetz(y=y)))
    zcr.append(np.mean(lr.feature.mfcc(y=y)))

In [44]:
mfccTarget = pd.DataFrame()
mfccTarget["mfcc"] = mfcc
mfccTarget["chroma_stft"] =chroma_stft
mfccTarget["chroma_cqt"] = chroma_cqt
mfccTarget["chroma_cens"] = chroma_cens
mfccTarget["rms"] = rms
mfccTarget["spectral_contrast"] = spectral_contrast
mfccTarget["spectral_bandwidth"] = spectral_bandwidth
mfccTarget["tonnetz"] =tonnetz
mfccTarget["zcr"] =zcr

In [30]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

In [39]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(np.array(mfcc).reshape(-1,1),data["Emotion"],test_size=0.30, random_state=42,shuffle=True)
rf = RandomForestClassifier(n_estimators = 100,random_state = 42)
rf.fit(X_train,y_train)
print("random forest algo result: ",rf.score(X_test,y_test))

random forest algo result:  0.16898148148148148


In [40]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(np.array(mfcc).reshape(-1,1),data["Emotion"],test_size=0.30, random_state=42,shuffle=True)
rf = RandomForestClassifier(n_estimators = 10000,random_state = 42)
rf.fit(X_train,y_train)
print("random forest algo result: ",rf.score(X_test,y_test))

random forest algo result:  0.16898148148148148


In [49]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(mfccTarget,data["Emotion"],test_size=0.30, random_state=42,shuffle=True)
rf = RandomForestClassifier(n_estimators = 100,random_state = 42)
rf.fit(X_train,y_train)
print("random forest algo result: ",rf.score(X_test,y_test))

random forest algo result:  0.42824074074074076


In [6]:
mfcc1 = []
chroma_stft1 = []
chroma_cqt1 = []
chroma_cens1 = []
rms1 = []
spectral_contrast1 = []
spectral_bandwidth1 = []
tonnetz1 = []
zcr1 = []

for i in range(0,len(audio_files)):
    
    y, sr = lr.load(audio_files[i],res_type="kaiser_fast",duration=3,offset=0.5)
    sr=np.array(sr)
    mfcc=(lr.feature.mfcc(y=y,hop_length=256,n_mfcc=13))
    if (259 > mfcc.shape[1]):
        pad_width = 259 - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        mfcc1.append(mfcc)
    else:
        mfcc1.append(mfcc)
    
mfccTarget1 = pd.DataFrame()
mfccTarget1["mfcc"] = mfcc1


In [129]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten,Dropout,MaxPooling1D,Activation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.asarray(mfccTarget1["mfcc"].tolist()),data["Emotion"].to_numpy(), test_size=0.30, random_state=42,shuffle=True)
model = Sequential()
model.add(Conv1D(256, 5,padding='same', input_shape=(13,259))) #1
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same')) #2
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same')) #3
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same')) #6
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(9)) #7
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20)

Train on 1008 samples, validate on 432 samples
Epoch 1/20
1008/1008 [==============================] - 4s 4ms/step - loss: 16.0326 - accuracy: 0.1488 - val_loss: 4.4000 - val_accuracy: 0.1829
Epoch 2/20
1008/1008 [==============================] - 2s 2ms/step - loss: 3.1931 - accuracy: 0.2212 - val_loss: 2.0139 - val_accuracy: 0.2593
Epoch 3/20
1008/1008 [==============================] - 2s 2ms/step - loss: 2.1246 - accuracy: 0.3036 - val_loss: 1.9088 - val_accuracy: 0.3171
Epoch 4/20
1008/1008 [==============================] - 2s 2ms/step - loss: 1.7427 - accuracy: 0.3810 - val_loss: 1.7349 - val_accuracy: 0.3310
Epoch 5/20
1008/1008 [==============================] - 2s 2ms/step - loss: 1.5098 - accuracy: 0.4554 - val_loss: 1.5181 - val_accuracy: 0.4468
Epoch 6/20
1008/1008 [==============================] - 2s 2ms/step - loss: 1.4402 - accuracy: 0.4692 - val_loss: 1.6370 - val_accuracy: 0.3912
Epoch 7/20
1008/1008 [==============================] - 2s 2ms/step - loss: 1.2881 - acc

In [132]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten,Dropout,MaxPooling1D,Activation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.asarray(mfccTarget1["mfcc"].tolist()),data["Emotion"].to_numpy(), test_size=0.30, random_state=42,shuffle=True)
model = Sequential()
model.add(Conv1D(256, 5,padding='same', input_shape=(13,259))) #1
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same')) #2
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same')) #3
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same')) #6
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(9)) #7
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100)

Train on 1008 samples, validate on 432 samples
Epoch 1/100
1008/1008 [==============================] - 2s 2ms/step - loss: 20.6616 - accuracy: 0.1429 - val_loss: 4.5163 - val_accuracy: 0.2338
Epoch 2/100
1008/1008 [==============================] - 2s 2ms/step - loss: 3.6909 - accuracy: 0.2312 - val_loss: 1.9859 - val_accuracy: 0.2477
Epoch 3/100
1008/1008 [==============================] - 2s 2ms/step - loss: 1.8777 - accuracy: 0.3234 - val_loss: 1.9474 - val_accuracy: 0.3171
Epoch 4/100
1008/1008 [==============================] - 2s 2ms/step - loss: 1.6384 - accuracy: 0.4167 - val_loss: 1.6515 - val_accuracy: 0.3843
Epoch 5/100
1008/1008 [==============================] - 2s 2ms/step - loss: 1.4562 - accuracy: 0.4702 - val_loss: 1.6037 - val_accuracy: 0.4398
Epoch 6/100
1008/1008 [==============================] - 2s 2ms/step - loss: 1.2566 - accuracy: 0.5427 - val_loss: 1.4553 - val_accuracy: 0.4560
Epoch 7/100
1008/1008 [==============================] - 2s 2ms/step - loss: 1.137

Epoch 57/100
1008/1008 [==============================] - 2s 2ms/step - loss: 0.0111 - accuracy: 0.9960 - val_loss: 2.3300 - val_accuracy: 0.6273
Epoch 58/100
1008/1008 [==============================] - 2s 2ms/step - loss: 0.0051 - accuracy: 0.9980 - val_loss: 2.2189 - val_accuracy: 0.6227
Epoch 59/100
1008/1008 [==============================] - 2s 1ms/step - loss: 0.0081 - accuracy: 0.9970 - val_loss: 2.4865 - val_accuracy: 0.6204
Epoch 60/100
1008/1008 [==============================] - 2s 2ms/step - loss: 0.0107 - accuracy: 0.9960 - val_loss: 2.3059 - val_accuracy: 0.6366
Epoch 61/100
1008/1008 [==============================] - 2s 2ms/step - loss: 0.0279 - accuracy: 0.9940 - val_loss: 2.3233 - val_accuracy: 0.5972
Epoch 62/100
1008/1008 [==============================] - 2s 2ms/step - loss: 0.0267 - accuracy: 0.9911 - val_loss: 2.4313 - val_accuracy: 0.6157
Epoch 63/100
1008/1008 [==============================] - 2s 2ms/step - loss: 0.0574 - accuracy: 0.9831 - val_loss: 2.4409 -

In [110]:
//Buna sonra bak
mfcc2 = []
chroma_stft2 = []
chroma_cqt2 = []
chroma_cens2 = []
rms2 = []
spectral_contrast2 = []
spectral_bandwidth2 = []
tonnetz2 = []
zcr2 = []

for i in range(0,len(audio_files)):
    
    y, sr = lr.load(audio_files[i],res_type="kaiser_fast",duration=3,offset=0.5)
    sr=np.array(sr)
    mfcc=(lr.feature.mfcc(y=y,hop_length=256,n_mfcc=13))
    if (259 > mfcc.shape[1]):
        pad_width = 259 - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        mfcc1.append(mfcc)
    else:
        mfcc1.append(mfcc)
    
mfccTarget2 = pd.DataFrame()
mfccTarget2["mfcc"] = mfcc2

(432,)

In [14]:
import logging
import os
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten,Dropout,MaxPooling1D,Activation
from sklearn.model_selection import train_test_split
  # local python class with Audio feature extraction (librosa)


X_train, X_test, y_train, y_test = train_test_split(np.asarray(mfccTarget1["mfcc"].tolist()),data["Emotion"].to_numpy(), test_size=0.30, random_state=42,shuffle=True)
model = Sequential()

model.add(LSTM(units=128, dropout=0.05, recurrent_dropout=0.35, return_sequences=True, input_shape=(13,259)))
model.add(LSTM(units=32,  dropout=0.05, recurrent_dropout=0.35, return_sequences=False))
model.add(Dense(units=9, activation="softmax"))
opt = Adam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.summary()

print("Training ...")
batch_size = 35  # num of training examples per minibatch
num_epochs = 400
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 13, 128)           198656    
_________________________________________________________________
lstm_6 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 297       
Total params: 219,561
Trainable params: 219,561
Non-trainable params: 0
_________________________________________________________________
Training ...
Train on 1008 samples, validate on 432 samples
Epoch 1/10
1008/1008 [==============================] - 4s 4ms/step - loss: 2.1000 - accuracy: 0.1796 - val_loss: 1.9811 - val_accuracy: 0.2199
Epoch 2/10
1008/1008 [==============================] - 1s 966us/step - loss: 1.8779 - accuracy: 0.3274 - val_loss: 1.9344 - val_accuracy: 0.2384
Epoch 3/10
1008/1008 [======

In [118]:
l=np.array([[[3,4],[9,0]],[[8,5],[7,6]]])

In [119]:
l.shape

(2, 2, 2)